In [ ]:
from llama_index.core import Settings 
from llama_index.llms.groq import Groq
from llama_index.core.tools import FunctionTool
from llama_index.core.agent import FunctionCallingAgentWorker
import os 

In [23]:
llm = Groq(
    model="llama-3.3-70b-versatile",
    api_key=os.environ.get("GROQ_API_KEY"),
)

In [25]:
def calcular_imposto_renda(rendimento: float) -> str:
    if rendimento <= 22847.76:
        return "Isento de Imposto de Renda"
    elif rendimento <= 33919.80:
        imposto = (rendimento - 22847.76) * 0.075
    elif rendimento <= 45012.60:
        imposto = (rendimento - 33919.80) * 0.15 + 826.29
    elif rendimento <= 55976.16:
        imposto = (rendimento - 45012.60) * 0.225 + 1428.14
    else:
        imposto = (rendimento - 55976.16) * 0.275 + 2203.91

    return f"Imposto de Renda devido: R$ {imposto:.2f}"

In [26]:
ferramenta_calculo_imposto = FunctionTool.from_defaults(
    fn=calcular_imposto_renda,
    name="calcular_imposto_renda",
    description=(
        "Calcula o Imposto de Renda devido com base no rendimento anual. "
        "Argumento: rendimento (float) - Rendimento anual em reais. "
        "Retorna uma mensagem indicando se é isento ou o valor do imposto a pagar."
    ),

)

In [ ]:
agent_worker_imposto = FunctionCallingAgentWorker.from_tools(
    llm=llm,
    tools=[ferramenta_calculo_imposto],
    verbose=True,
    allow_parallel_tool_calls=True, # Permite chamadas paralelas de ferramentas
)